In [1]:
from qcodes.instrument.specialized_parameters import ElapsedTimeParameter
from qcodes.dataset.plotting import plot_dataset,plot_by_id
from qcodes.logger.logger import start_all_logging
from tqdm.notebook import tqdm
from qcodes.loops import Loop
from time import sleep

import matplotlib.pyplot as plt
import qcodes as qc
import numpy as np
import datetime
import time
import json
import sys
import os

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
sys.path.append('C:\QCodes_Driver_And_Scripts\Qcodes-drivers')

from qcodes.instrument_drivers.yokogawa.GS200 import GS200
from Agilent_E8257D import E8257D
from RS_SGS_100A import SGS100A
from VNA_RS import VNA

try:
    VNA = VNA('ZNB20', 'TCPIP0::192.168.10.2::inst0::INSTR', init_s_params=False,terminator='\n')
except KeyError as er:
    VNA.close_all() #Disconnect and irreversibly tear down the instrument
    VNA = VNA('ZNB20', 'TCPIP0::192.168.10.2::inst0::INSTR',init_s_params=False,terminator='\n')
    
try:
    I_source = GS200("gs200", address='TCPIP::192.168.10.3::INSTR', terminator="\n")
except KeyError as er:
    GS200.close_all() #Disconnect and irreversibly tear down the instrument
    I_source = GS200("gs200", address='TCPIP::192.168.10.3::INSTR', terminator="\n")

try:
    pump_source=SGS100A('pump_source','TCPIP0::192.168.10.56::inst0::INSTR')
except KeyError as er:
    SGS100A.close_all() #Disconnect and irreversibly tear down the instrument
    pump_source=SGS100A('pump_source','TCPIP0::192.168.10.56::inst0::INSTR')

try:
    MW_source=E8257D('mw_source','TCPIP0::192.168.10.70::inst0::INSTR')
except KeyError as er:
    E8257D.close_all() #Disconnect and irreversibly tear down the instrument
    MW_source=E8257D('mw_source','TCPIP0::192.168.10.70::inst0::INSTR')

Connected to: Rohde-Schwarz ZNB20-2Port (serial:1311601062101578, firmware:2.80) in 0.06s
Connected to: YOKOGAWA GS210 (serial:90Y438440, firmware:2.02) in 0.02s
Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/114045, firmware:4.2.76.0-4.30.046.295) in 0.06s
Connected to: Agilent Technologies E8257D (serial:US45470698, firmware:C.06.28) in 0.04s


In [4]:
def measure(power_vec, exp, station):

    VNA.rf_on()

    meas = qc.Measurement(exp=exp, station=station)
    meas.register_parameter(pump_source.power)
    meas.register_custom_parameter(name='twpa_gain',paramtype='array',setpoints=[pump_source.power])
    
    
    meas.write_period = 10

    with meas.run() as datasaver:

        run_id=datasaver.dataset.run_id
        qc.load_by_run_spec(captured_run_id=run_id).add_metadata('parameter_snap',json.dumps(parameter_snap))

        for power in tqdm(power_vec):
            
            pump_source.power(power)
            
            pump_source.status('on')
            get_p = float(pump_source.power.get())
            
            get_v_on = VNA.channels.S21.trace_mag_phase.get()[0]
            
            pump_source.status('off')
            
            get_v_off = VNA.channels.S21.trace_mag_phase.get()[0]

            datasaver.add_result(('twpa_gain', get_v_on/get_v_off),(pump_source.power,get_p))

In [5]:
start_all_logging()

# Create a station
station = qc.Station()
station.add_component(VNA)
station.add_component(MW_source)
station.add_component(pump_source)
station.add_component(I_source)

station.snapshot()
station.components

# Experiment details
user='DF'
date=datetime.datetime.today().strftime('%Y_%m_%d')
description='twpa_on_off_vs_pump_power'
database_name = date+"_"+user+"_"+description

script_dir=os.path.dirname(os.path.abspath("__file__"))
data_dir=os.path.join('C:\\QCodes_Driver_And_Scripts\\Data_QCodes\\Dorian\\'+date)

try :
    os.mkdir(data_dir)
except FileExistsError:
    pass

data_dir=data_dir +'\\'+description

try :
    os.mkdir(data_dir)
except FileExistsError:
    pass

qc.initialise_or_create_database_at(data_dir+'\\'+database_name+'.db')
qc.config.core.db_location

parameter_snap={}

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\nicolas.ROCH\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\nicolas.ROCH\.qcodes\logs\230110-26220-qcodes.log


2023-01-10 10:38:31,238 ¦ qcodes.instrument.instrument_base ¦ WARNING ¦ instrument_base ¦ snapshot_base ¦ 292 ¦ [mw_source(E8257D)] Snapshot: Could not update parameter: sweepmode
2023-01-10 10:38:36,856 ¦ qcodes.instrument.instrument_base ¦ WARNING ¦ instrument_base ¦ snapshot_base ¦ 292 ¦ [gs200_program(GS200Program)] Snapshot: Could not update parameter: trigger


In [6]:
# coil parameters
current = -2.756#mA

# Snapshot parameters
parameter_snap['flux']={'current': current}

#VNA parameters
start_freq   = 2.
end_freq     = 9.
VNA_power    = -20
IF_bandwidth = 10
Averages     = 1
points_VNA   = 2001
trace_time   = points_VNA / IF_bandwidth * Averages

print('\n time for one trace : {} s'.format(datetime.timedelta(seconds = trace_time)))

# Snapshot parameters
parameter_snap['vna'] = {'start_freq':start_freq, 'end_freq':end_freq,'VNA_power':VNA_power, 'IF_bandwidth':IF_bandwidth,'Averages':Averages, 'points':points_VNA}

# twpa pump parameters
pump_freq    = 4.998              
pump_power   = np.array([0.5])          
twpa_current = -0.5      

# Snapshot parameters
parameter_snap['twpa_pump'] = {'freq':pump_freq,'current' : twpa_current}

#MW source parameters
src_frequency  = 9.155
src_power      = -20 
src_status     = 'off'

# Snapshot parameters
parameter_snap['MW_source'] = {'freq':src_frequency,'power':src_power,'status':src_status}

print('\ntotal time : {}'.format(datetime.timedelta(seconds = 2*len(pump_power)*trace_time)))


 time for one trace : 0:03:20.100000 s

total time : 0:06:40.200000


# setting up instruments

In [7]:
if I_source.output()=='on':
    I_source.output('off')

I_source.source_mode('CURR')
I_source.range(10e-3)
I_source.current(current*1.e-3)
I_source.output('on')

VNA.add_channel('S21',vna_parameter='S21')
VNA.channels.S21.format('dB')
VNA.channels.S21.bandwidth(IF_bandwidth)
VNA.channels.S21.power(VNA_power)
VNA.channels.S21.avg(Averages)
VNA.channels.npts(points_VNA)
VNA.channels.start(start_freq*1e9)
VNA.channels.stop(end_freq*1e9)

MW_source.frequency(src_frequency*1e9)
MW_source.power(src_power)
MW_source.status(src_status.upper())

pump_source.frequency(pump_freq*1.e9)
pump_source.power(pump_power[0])

In [8]:
# Load experiment
exp_name = 'twpa_on_off_vs_pump_power'

sample_name = 'B5_G3_GALV_SQ200_1ST'

exp=qc.load_or_create_experiment(experiment_name=exp_name,sample_name=sample_name)

measure(pump_power,exp=exp, station=station)

# Shutdown devices
VNA.rf_off()
VNA.cont_meas_off()
MW_source.status('OFF')
pump_source.status('off')
I_source.reset()

Starting experimental run with id: 1. 


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
pump_source.power.get()

'0.5'